In [1]:
from src import API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Use device name: NVIDIA GeForce RTX 4090
Unpack Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.240528000213.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240528094134.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240529072117.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240529231835.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240531070145.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240531223956.tar']
Sun Jun  2 22:03:20 2024 : download since!
Connection and Factor preparation finished!
Sun Jun  2 22:03:20 2024 : sellside/haitong.hf_factors from 20240525 to 20240602, total 1 periods
Start sellside/haitong.hf_factors:20240525-20240602 
Done sellside/haitong.hf_factors:20240525-20240602, cost 8.4 Secs
Sun Jun  2 22:03:28 2024 : sellside/haitong.dl_factors from 20240525 to 20240602, total 1 periods
Start sellside/haitong.dl_factors:20240525-20240602 
Done sellside/haitong.dl_factors:20240525-20240602, cost 225.3 Secs
S

In [2]:
from src import API
API.Predictor.update_factors()

Load  2 DataBlocks...... finished! Cost 0.11 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.15 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


In [2]:
from src import API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
